In [2]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit,fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"


# mpl.rcParams.update({"axes.grid" : True})
import util_old as yu
yu.flag_fast=False

In [3]:
enss=['a','b','c','d']
ens2full={'a':'cA2.09.48','b':'B64','c':'C80','d':'D96'}
ens2a={'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692} # fm

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

data={}
path='/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt/'
for ens in enss:
    data[ens]={}
    with h5py.File(path+ens2full[ens]+'.h5') as f:
        for ky in f['data'].keys():
            data[ens][ky]=f['data'][ky][:]
    

In [14]:
# cosh mass

def run():
    cases=['piC','pi0_conn','pi0_disc','pi0']
    case2color={'piC':'r','pi0_conn':'b','pi0_disc':'purple','pi0':'g'}
    case2keys={'piC':['piC'],'pi0_conn':['pi0_conn'],'pi0_disc':['pi0_disc'],'pi0':['pi0_conn','pi0_disc']}

    for ens in enss:
        fig, axs = yu.getFigAxs(1,1,scale=4)
        
        fig.suptitle(ens2full[ens],fontsize=40)

        xUnit=ens2a[ens]
        yUnit=ens2aInv[ens]
        
        timeT=data[ens]['piC'].shape[1]
        tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))
        
        # tau={'a':6,'b':7,'c':8,'d':9}[ens]
        tau={'a':4,'b':13,'c':15,'d':18}[ens]
        
        for case in cases:
            color=case2color[case]
            keys=case2keys[case]
            
            vevQ = 'pi0_disc' in keys
            t=np.sum([data[ens][key] for key in keys],axis=0)
            t=t[:,:,0]
            t=[t,data[ens]['vev']]

            def func(dat):
                t,tVEV=dat
                t=np.mean(t,axis=0)
                if vevQ:
                    t-=np.mean(tVEV)**2
                t=np.real(t)
                t=(t+np.roll(np.flip(t),1,axis=0))/2

                tP=np.roll(t,-tau,axis=0)
                tM=np.roll(t,+tau,axis=0)
                t=np.arccosh( (tP+tM)/(2*t) )/tau

                return t
                
            mean,err,cov=yu.jackknife(t,func)
            tmin=1;tmax=timeT
            tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
            
            irow=icol=0
            axs[irow,icol].errorbar(np.arange(tmin,tmax)*xUnit,tMean*yUnit,tErr*yUnit,color=color,label=(case,'{:.2f}fm'.format(tau*xUnit)))
            
            
        # color='orange'
        # # (tau,t0)={'a':(6,6),'b':(7,7),'c':(8,8),'d':(9,9)}[ens]
        # (tau,t0)={'a':(4,4),'b':(13,13),'c':(15,15),'d':(18,18)}[ens]
        # for case in ['pi0']:
        #     keys=case2keys[case]
            
        #     vevQ = 'pi0_disc' in keys
        #     t=np.sum([data[ens][key] for key in keys],axis=0)
        #     t=t[:,:,0]
        #     t=[t,data[ens]['vev']]

        #     def func(dat):
        #         t,tVEV=dat
        #         t=np.complex256(t)
        #         t=np.mean(t,axis=0)
        #         t=np.complex128(t)
        #         if vevQ:
        #             t-=np.mean(tVEV)**2
        #         t=np.real(t)
        #         t=(t+np.roll(np.flip(t),1,axis=0))/2
                
        #         t=(np.roll(t,-tau,axis=0)-np.roll(t,-t0-tau,axis=0))/(t-np.roll(t,-t0,axis=0))
        #         t=np.array([fsolve(lambda m:((tfunc_C(m,xt+tau)-tfunc_C(m,xt+tau+t0))/(tfunc_C(m,xt)-tfunc_C(m,xt+t0)))-ele, 0.0593)[0] for xt,ele in enumerate(t)])
        #         t=[ele if ele!=0.0593 else np.NaN for ele in t]
        #         return t
                
        #     mean,err,cov=yu.jackknife(t,func)
        #     tmin=1;tmax=timeT
        #     tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]

        #     irow=icol=0
        #     axs[irow,icol].errorbar((np.arange(tmin,tmax)+(tau+t0)/2)*xUnit,tMean*yUnit,tErr*yUnit,color=color,label=('{:.2f}fm'.format(tau*xUnit),'{:.2f}fm'.format(t0*xUnit)))
        
        axs[irow,icol].legend(fontsize=40,loc='upper center')
        axs[irow,icol].set_xlim([0,12])
        axs[irow,icol].set_ylim([0,400])
        
        xlim=[0,20]
        piC_exp=139.57039; pi0_exp=134.9768
        t_mean,t_err=np.array([(piC_exp+pi0_exp)/2,(piC_exp-pi0_exp)/2])
        axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
        
        plt.tight_layout()
        plt.savefig(f'fig/2_{ens}.pdf')
        plt.close()
        
run()

In [ ]:
# c2

def run():
    fig, axs = yu.getFigAxs(1,1,scale=4)
    irow=icol=0
   
    for ens in enss:
        
        ens2color={'a':'orange','b':'b','c':'g','d':'r'}
        xUnit=ens2a[ens]
        yUnit=1e-6
        
        timeT=data[ens]['piC'].shape[1]
        tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))
        
        dat_C=data[ens]['piC']
        dat_0=data[ens]['pi0_conn']+data[ens]['pi0_disc']
        dat_vl=data[ens]['vev']
        dat=[dat_C[:,:,0],dat_0[:,:,0],dat_vl]
        
        tau=1
        
        def func(dat):
            t_C,t_0,t_vl=dat
            
            t=t_C
            t=np.mean(t,axis=0)
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            tP=np.roll(t,-tau,axis=0)
            tM=np.roll(t,+tau,axis=0)
            t=np.arccosh( (tP+tM)/(2*t) )/tau
            t_C=t
            
            t=t_0
            t=np.complex256(t)
            t=np.mean(t,axis=0)
            t=t-np.mean(np.complex256(t_vl))**2
            t=np.complex128(t)
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            tP=np.roll(t,-tau,axis=0)
            tM=np.roll(t,+tau,axis=0)
            t=np.arccosh( (tP+tM)/(2*t) )/tau
            t_0=t
            
            t=(t_0**2-t_C**2)*(ens2aInv[ens]**2)
            # t=(t_0**2-t_C[{'a':10,'b':13,'c':15,'d':18}[ens]]**2)*(ens2aInv[ens]**2)
            # t=(t_0**2-t_C[20]**2)*(ens2aInv[ens]**2)
            return t
            
        mean,err,cov=yu.jackknife(dat,func)
        tmin=1;tmax=timeT
        tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]

        axs[irow,icol].errorbar((np.arange(tmin,tmax))*xUnit,tMean*yUnit,tErr*yUnit,color=ens2color[ens],label=ens2full[ens])
        
        func(dat)
    
    xlim=[0,20]
    # axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
    axs[0,0].hlines(0,xlim[0],xlim[1],color='black',linestyle='--')
    axs[0,0].legend(fontsize=40,loc='upper center')
    axs[0,0].set_xlim([0,2.5])
    axs[0,0].set_ylim([-0.32,0.32])
    # axs[0,0].set_ylim([-0.02,0.02])
    
    # xlim=[0,20]
    # piC_exp=139.57039; pi0_exp=134.9768
    # t_mean,t_err=np.array([(piC_exp+pi0_exp)/2,(piC_exp-pi0_exp)/2])
    # axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
    
    plt.tight_layout()
    # plt.savefig(f'fig/c2_C20_zoom_tau{tau}.pdf')
    # plt.close()
    
run()

In [ ]:
# mass diff

def run():
    fig, axs = yu.getFigAxs(1,1,scale=4)
    irow=icol=0
   
    for ens in enss:
        
        ens2color={'a':'orange','b':'b','c':'g','d':'r'}
        xUnit=ens2a[ens]
        yUnit=1e-6
        yUnit=ens2aInv[ens]
        
        timeT=data[ens]['piC'].shape[1]
        tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))
        
        dat_C=data[ens]['piC']
        dat_0=data[ens]['pi0_conn']+data[ens]['pi0_disc']
        dat_vl=data[ens]['vev']
        dat=[dat_C[:,:,0],dat_0[:,:,0],dat_vl]
        
        tau=10
        
        def func(dat):
            t_C,t_0,t_vl=dat
            
            t=t_C
            t=np.mean(t,axis=0)
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            tP=np.roll(t,-tau,axis=0)
            tM=np.roll(t,+tau,axis=0)
            t=np.arccosh( (tP+tM)/(2*t) )/tau
            t_C=t
            
            t=t_0
            t=np.complex256(t)
            t=np.mean(t,axis=0)
            t=t-np.mean(np.complex256(t_vl))**2
            t=np.complex128(t)
            t=np.real(t)
            t=(t+np.roll(np.flip(t),1,axis=0))/2
            tP=np.roll(t,-tau,axis=0)
            tM=np.roll(t,+tau,axis=0)
            t=np.arccosh( (tP+tM)/(2*t) )/tau
            t_0=t
            
            t=t_C-t_0
            # t=(t_0**2-t_C**2)*(ens2aInv[ens]**2)
            # t=(t_0**2-t_C[{'a':10,'b':13,'c':15,'d':18}[ens]]**2)*(ens2aInv[ens]**2)
            # t=(t_0**2-t_C[20]**2)*(ens2aInv[ens]**2)
            return t
            
        mean,err,cov=yu.jackknife(dat,func)
        tmin=1;tmax=timeT
        tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]

        axs[irow,icol].errorbar((np.arange(tmin,tmax))*xUnit,tMean*yUnit,tErr*yUnit,color=ens2color[ens],label=ens2full[ens])
        
        func(dat)
    
    xlim=[0,20]
    # axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
    axs[0,0].hlines(0,xlim[0],xlim[1],color='black',linestyle='--')
    axs[0,0].legend(fontsize=40,loc='upper center')
    axs[0,0].set_xlim([0,2.5])
    axs[0,0].set_ylim([-0.32,0.32])
    axs[0,0].set_ylim([-0.02,0.02])
    axs[0,0].set_ylim([-10,50])
    
    # xlim=[0,20]
    # piC_exp=139.57039; pi0_exp=134.9768
    # t_mean,t_err=np.array([(piC_exp+pi0_exp)/2,(piC_exp-pi0_exp)/2])
    # axs[0,0].fill_between(xlim,t_mean-t_err,t_mean+t_err,color='grey',alpha=0.2)
    
    plt.tight_layout()
    # plt.savefig(f'fig/c2_C20_zoom_tau{tau}.pdf')
    # plt.close()
    
run()